# Calculating Weighted Mean 

UXarray supports weighting means based on geometries, such as face areas. This user-guide section will discuss how to calculate variable means them using unstructured grid geometries as weights. Below showcases two examples using weighted mean with grid face areas or edge lengths as weights. 

Currently, `weighted-mean` does not support nodes as weights. 

In [1]:
import uxarray as ux
import xarray as xr
import cartopy.crs as ccrs

import warnings

warnings.filterwarnings("ignore")

## Data

Data used in this section is a 3-random-variable dataset on a quad hexagon mesh mapped to the nodes, edges, and faces.

In [28]:
grid_path = "../../test/meshfiles/ugrid/quad-hexagon/grid.nc"
quad_hex_data_path_edge_centered = "../../test/meshfiles/ugrid/quad-hexagon/random-edge-data.nc"
quad_hex_data_path_face_centered = "../../test/meshfiles/ugrid/quad-hexagon/random-face-data.nc"

uxds_face = ux.open_mfdataset(grid_path, quad_hex_data_path_face_centered)
uxds_edge = ux.open_mfdataset(grid_path, quad_hex_data_path_edge_centered)

The figure below visualizes the dataset. 

In [10]:
(
    uxds.uxgrid.plot(line_color="black")
    * uxds_edge
    .plot(
        cmap="inferno", size=150, marker="square", clabel=None, tools=["hover"]
    )
    .relabel("Edge Data")
    * uxds_face
    .plot(
        cmap="inferno", size=150, marker="triangle", clabel=None, tools=["hover"]
    )
    .relabel("Face Data")
).opts(legend_position="top_right")

ValueError: UxDataset.plot cannot be called directly. Use an explicit plot method, e.g uxds.plot.scatter(...)

## Weighted Mean based on Face Areas
Here we first look at the data values on each face and the faces' respsective areas. 

In [21]:
uxds_face['random_data_face'].values

array([4.53649447, 3.31347744, 6.20654387, 2.48819151])

In [31]:
uxds_face.uxgrid.face_areas.data

array([1.95941760e-05, 1.96026212e-05, 1.96001041e-05, 1.95966022e-05])

We can simply call `.weighted_mean()` on the UXDataArray to compute the weighted mean. The differences between the weighted mean and the regular mean is small since the area differences across the faces are small. 

In [36]:
result = uxds_face['random_data_face'].weighted_mean()
result.values

array(4.13619375)

In [18]:
unweighted_result = uxds_face['random_data_face'].mean()
unweighted_result.values

array(4.13617682)

## Weighted Mean Based on Edge Length

Here we show the similar steps but for edge-centered data and the edge lengths. 

In [22]:
uxds_edge['random_data_edge'].values

array([8.01802324, 9.72507993, 6.6914741 , 7.32080453, 3.30975404,
       0.22819568, 0.06583025, 6.35602489, 6.68668242, 9.7668741 ,
       6.30413784, 6.89570468, 1.90836517, 0.57331484, 0.81162917,
       7.37824547, 2.2410142 , 1.46405388, 1.90183779])

In [32]:
uxds_edge.uxgrid.edge_node_distances.data

array([0.00302971, 0.00276214, 0.00244141, 0.00241687, 0.00276214,
       0.00274047, 0.00302971, 0.00244141, 0.00276214, 0.00304932,
       0.00276214, 0.00304932, 0.0024657 , 0.00304932, 0.00276214,
       0.0024657 , 0.00244141, 0.00278363, 0.00304932], dtype=float32)

The differences between weighted and unweighted mean is more drastic (~0.1 value difference) since the edge lengths have a larger variance. 

In [33]:
result = uxds_edge['random_data_edge'].weighted_mean()
result.values

array(4.58431796)

In [34]:
unweighted_result = uxds_edge['random_data_edge'].mean()
unweighted_result.values

array(4.61300243)

## Custom Weights

While custom weights are supported in `weighted-mean`, it is crucial that the weight inputs are mapped on the same unstructured grid as the data for accurate calculations. 

Data can be stored in various configurations in the unstructured grid. The figure below shows two possible configuration for face-centered data and edge-centered data can be stored for this particular grid.  




<img src="../_static/examples/weightedmean/edge_face_weightedmean.png" width="600" alt="Optional Alt Text">